In [ ]:
import numpy as np
import cv2
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()


#images -> black and white, with no shades of gray
def binarize(image):
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    return binary_image

# Split image into 3x3 sub-images (blocks)
def split_image(image, num_blocks=9):
    sub_images = []
    h, w = image.shape
    # ensure that the image is divided evenly into a 3x3 grid of blocks
    block_h = h // 3
    block_w = w // 3
    for i in range(3):
        for j in range(3):
            #Slicing the input image to extract a block
            block = image[i*block_h:(i+1)*block_h, j*block_w:(j+1)*block_w]
            sub_images.append(block)
    return sub_images

# Extract centroids from each block of the image
def extract_centroids(image, num_blocks):
    block_size = image.shape[0] // num_blocks
    centroids = []
    for i in range(num_blocks):
        for j in range(num_blocks):
            block = image[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            #find the indices of the pixels in the block that have the same value as the maximum value
            centroid_x = np.mean(np.where(block == np.max(block))[0])
            #[1]-> extracts the column indices of the brightest pixels within the block
            centroid_y = np.mean(np.where(block == np.max(block))[1])
            centroids.append([centroid_x, centroid_y])
    return np.array(centroids).flatten()



# Compute chain code from contour
def compute_chain_code(contour):
    chain_code = []
    # Define direction encoding (E=0, NE=1, N=2, NW=3, W=4, SW=5, S=6, NE=7)
    directions = {0: 0, 45: 1, 90: 2, 135: 3, 180: 4, 225: 5, 270: 6, 315: 7}

    #this check ensures that the contour has enough points to compute a chain code
    if len(contour) < 2:
        return chain_code

    # Iterate over contour points except for the last one because we need to compute the displacement vector between consecutive points
    for i in range(len(contour) - 1):
        # Compute the displacement vector between consecutive points
        dx = contour[i + 1][0][0] - contour[i][0][0]
        dy = contour[i + 1][0][1] - contour[i][0][1]

        # Compute the direction angle (in radians )
        angle = np.arctan2(dy, dx) * 180 / np.pi

        # Normalize angle to [0, 360] degrees
        #If the angle is negative, adding 360 to shift it into positive
        #If the angle is positive but exceeds 360, modulus bring it back into the range
        angle = (angle + 360) % 360

        # Round angle to the nearest multiple of 45 degrees
        nearest_angle = round(angle / 45) * 45

        # Map angle to direction encoding
        direction = directions[nearest_angle]

        # Append direction to the chain code
        chain_code.append(direction)

    return chain_code

# Extract contours from binarized image
def extract_contours(binarized_image):
    contours, _ = cv2.findContours(binarized_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# Extract chain codes from contours of sub-images
def extract_chain_codes(sub_images, max_length=100):
    chain_codes = []
    for sub_image in sub_images:
        # Extract contours from sub-image
        contours = extract_contours(sub_image)
        if len(contours) > 0:
            # Select the largest contour
            contour = max(contours, key=cv2.contourArea)
            # Compute chain code for the contour
            chain_code = compute_chain_code(contour)
            # Pad or truncate chain code to ensure consistent length
            if len(chain_code) < max_length:
                chain_code += [0] * (max_length - len(chain_code))  # Pad with zeros
            else:
                chain_code = chain_code[:max_length]  # Truncate
            # Append chain code to the list
            chain_codes.append(chain_code)
        else:
            # If no contour found, append zeros for padding
            chain_codes.append([0] * max_length)
    return chain_codes

# Extract features from images
def extract_features(images, feature_type='centroid'):
    features = []
    for image in images:
        # Binarize the image
        binary_image = binarize(image)
        # Split the binary image into sub-images
        sub_images = split_image(binary_image)
        # Extract features based on the selected feature type
        if feature_type == 'centroid':
            features.append(extract_centroids(image, num_blocks=9))
        elif feature_type == 'chain_code':
            features.append(np.concatenate(extract_chain_codes(sub_images)))
    return np.array(features)

# Implement K-means clustering
def kmeans(X, k=10, max_iters=100):
    # Initialize centroids randomly
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]

    # Iterate until convergence or maximum iterations reached
    for _ in range(max_iters):
        # Compute distances between each data point and centroids
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)

        # Assign each data point to the nearest centroid
        labels = np.argmin(distances, axis=1)

        # Compute new centroids as the mean of data points assigned to each centroid
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])

        # Check for convergence
        if np.all(centroids == new_centroids):
            break

        # Update centroids
        centroids = new_centroids

    # Return final centroids and labels
    return centroids, labels


# Split data into training/testing sets with percentage 80/20
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Extract centroid features for all samples in training set
X_train_centroid_features = extract_features(X_train, feature_type='centroid')

# Extract chain code features for all samples in training set
X_train_chain_code_features = extract_features(X_train, feature_type='chain_code')

# Implement K-mean clustering algorithm on centroid features
centroids_centroid, labels_centroid = kmeans(X_train_centroid_features, k=10)

# Implement K-mean clustering algorithm on chain code features
centroids_chain_code, labels_chain_code = kmeans(X_train_chain_code_features, k=10)

# Compare the performance of k-means with centroid features and chain code features
def compare_performance(labels, y_train):
    class_counts = np.zeros((10, 10))  # 10 classes, 10 clusters
    for i in range(len(y_train)):
        class_counts[y_train[i]][labels[i]] += 1
    return class_counts

centroid_performance = compare_performance(labels_centroid, y_train)
chain_code_performance = compare_performance(labels_chain_code, y_train)

print("Performance of K-means with centroid features:")
print(centroid_performance)
print("\nPerformance of K-means with chain code features:")
print(chain_code_performance)

"""
def assign_to_centroids(data, centroids):
    distances = np.linalg.norm(data[:, np.newaxis, :] - centroids, axis=2)
    return np.argmin(distances, axis=1)

def assign_majority_class(cluster_labels, true_labels, k):
    predicted_labels = np.zeros_like(cluster_labels)
    for i in range(k):
        mask = (cluster_labels == i)
        majority_class = np.argmax(np.bincount(true_labels[mask]))
        predicted_labels[mask] = majority_class
    return predicted_labels

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Extract centroid features for all samples in validation set
X_val_centroid_features = extract_features(X_val, feature_type='centroid')

# Extract chain code features for all samples in validation set
X_val_chain_code_features = extract_features(X_val, feature_type='chain_code')

# Assign cluster labels to validation data using centroids
val_cluster_labels_centroid = assign_to_centroids(X_val_centroid_features, centroids_centroid)

# Assign cluster labels to validation data using chain code
val_cluster_labels_chain_code = assign_to_centroids(X_val_chain_code_features, centroids_chain_code)

# Assign predicted labels based on majority class in each cluster for centroid features
val_predicted_labels_centroid = assign_majority_class(val_cluster_labels_centroid, y_val, 10)

# Assign predicted labels based on majority class in each cluster for chain code features
val_predicted_labels_chain_code = assign_majority_class(val_cluster_labels_chain_code, y_val, 10)

# Calculate accuracy for centroid features
val_accuracy_centroid = calculate_accuracy(y_val, val_predicted_labels_centroid)

# Calculate accuracy for chain code features
val_accuracy_chain_code = calculate_accuracy(y_val, val_predicted_labels_chain_code)

print("Validation Accuracy with centroid features:", val_accuracy_centroid)
print("Validation Accuracy with chain code features:", val_accuracy_chain_code)

# Compare the performance of k-means with centroids features and chain code features
def compare_performance_kmeans(centroid_performance, chain_code_performance):
    class_counts_real = np.sum(centroid_performance, axis=1)
    class_counts_centroid = np.sum(centroid_performance, axis=0)
    class_counts_chain_code = np.sum(chain_code_performance, axis=0)
    better_performance = np.less(np.abs(class_counts_real - class_counts_centroid), np.abs(class_counts_real - class_counts_chain_code))
    return better_performance

better_performance_kmeans = compare_performance_kmeans(centroid_performance, chain_code_performance)

print("\nComparison of K-means performance with centroid and chain code features:")
print(better_performance_kmeans)
"""

Performance of K-means with centroid features:
[[5.000e+00 6.000e+01 5.600e+01 3.200e+01 3.021e+03 1.200e+01 8.300e+01
  4.700e+01 1.241e+03 1.910e+02]
 [9.000e+00 7.000e+00 5.000e+00 2.400e+01 0.000e+00 5.236e+03 8.600e+01
  1.900e+01 2.900e+01 5.000e+00]
 [7.400e+01 2.030e+02 1.520e+02 2.900e+01 7.800e+01 5.600e+02 3.138e+03
  1.810e+02 3.160e+02 5.300e+01]
 [3.100e+01 3.210e+03 1.410e+02 6.900e+01 3.600e+01 2.240e+02 1.850e+02
  5.110e+02 4.640e+02 4.100e+01]
 [1.085e+03 7.000e+00 1.509e+03 1.347e+03 3.000e+01 1.810e+02 1.290e+02
  4.400e+01 2.530e+02 8.100e+01]
 [1.480e+02 9.720e+02 3.160e+02 1.740e+02 1.180e+02 2.510e+02 5.600e+01
  7.770e+02 1.372e+03 1.330e+02]
 [1.200e+01 1.900e+01 8.200e+01 1.100e+01 5.600e+01 1.400e+02 1.380e+02
  3.800e+01 1.655e+03 2.590e+03]
 [2.262e+03 1.600e+01 6.260e+02 1.648e+03 3.500e+01 2.520e+02 6.600e+01
  1.300e+01 3.800e+01 1.000e+01]
 [1.520e+02 6.780e+02 2.680e+02 9.200e+01 1.200e+01 3.660e+02 3.250e+02
  2.597e+03 1.590e+02 4.200e+01]
 [1.211e